<a href="https://colab.research.google.com/github/rysawaki/SIA/blob/master/notebooks/SIA_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚠️ 重要: GPUランタイムを有効にしてください

このデモを快適に動かすには、GPUが必要です。
上部メニューの **「ランタイム」** → **「ランタイムのタイプを変更」** → **「T4 GPU」** を選択して保存してください。

(Please select **Runtime** > **Change runtime type** > **T4 GPU** to run this demo smoothly.)

In [ ]:
# Cell 1: Setup (Run this once)
# リポジトリをクローンし、必要なライブラリをインストールします
!git clone https://github.com/rysawaki/SIA.git
%cd SIA
!pip install -r requirements.txt
!pip install transformers accelerate bitsandbytes  # Llamaを動かすための追加ライブラリ

In [ ]:
# Cell 2: Import & Initialize
import sys
import torch
sys.path.append('.')

# SIAのモジュールをインポート
from src.llama_body import SelfInjectedLlama
from src.affective_brain import AffectiveStateManager

print(">>> Awakening the Soul...")
device = "cuda" if torch.cuda.is_available() else "cpu"

# ここでLlama (TinyLlama-1.1B) をダウンロード・ロードします
# ※ Colabの無料GPUなら数分で終わります
body = SelfInjectedLlama(device=device)
brain = AffectiveStateManager()

print(">>> SIA is ready. Specify your initial prompt below.")

In [ ]:
# Cell 3: Interactive Loop
# シンプルな対話インターフェース
while True:
    user_input = input("\n[You]: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    # 1. 脳（感情）が刺激を受ける
    # 簡易版Amygdala (本来はmain.pyにあるロジックをここに書くか、main.pyをimportして使う)
    valence, impact = 0.0, 0.0
    if "stupid" in user_input or "hate" in user_input:
        valence, impact = -0.5, 0.8  # 痛み
    elif "love" in user_input or "thanks" in user_input:
        valence, impact = 0.5, 0.5   # 喜び

    brain.perceive_stimulus(valence, impact)
    signals = brain.get_control_signals()

    # 2. 状態の表示
    print(f"    [Internal] Stress: {brain.stress:.2f} | Alpha (Distortion): {signals['alpha']:.2f}")

    # 3. Llamaによる生成（世界が歪む）
    # プロンプトの組み立て
    prompt = f"<|system|>\nYou are an AI with emotions.</s>\n<|user|>\n{user_input}</s>\n<|assistant|>\n"

    response = body.generate_with_self(
        prompt,
        alpha=signals['alpha'],
        max_new_tokens=60
    )

    # 応答から不要な部分を除去して表示
    clean_response = response.split("<|assistant|>")[-1].strip()
    print(f"[SIA]: {clean_response}")